# Creating the mdoel

In [2]:
import pandas as pd
# import numpy as np
import re
import nltk
import random

In [4]:
dataset = pd.read_csv('documents.csv')

In [5]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data files
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
all_stopwords = stopwords.words('english')

# Process text
corpus = []
length = len(dataset['text'])
for i in range(length):
    review = re.sub('[^a-zA-Z\']', ' ', dataset['text'][i])  # Keep apostrophes
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

# print(corpus[:10])  # Print the first 10 processed texts for verification
# print(all_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pushpgarg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pushpgarg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/pushpgarg/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(corpus)
y = dataset['label']

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(bootstrap = False, criterion = 'gini', max_depth = None, max_features =  'log2', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50)
classifier.fit(X_train, y_train)



RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=50)

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 8  0]
 [ 0 12]]


1.0

In [11]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Assuming X and y are your features and target labels
# Replace X and y with your actual data

# Initialize the classifier
classifier = RandomForestClassifier()

# Initialize StratifiedKFold with k=5
cv = StratifiedKFold(n_splits=10, shuffle=True)

# Perform k-fold cross-validation
accuracy_scores = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy', n_jobs=-1)

# Print the accuracy scores for each fold
print("Cross-validation accuracy scores:")
print(accuracy_scores)

# Print the mean and standard deviation of the accuracy scores
print(f"Mean accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Standard deviation: {np.std(accuracy_scores):.4f}")


Cross-validation accuracy scores:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean accuracy: 1.0000
Standard deviation: 0.0000


In [12]:
# Test texts for invoices and contracts
test_texts = [
    # Invoice examples
    "Invoice #12345 issued on 15-09-2023 for $5000.",
    "Payment due for invoice #789 on 01-11-2023. Total amount: $1500.",
    "Kindly remit payment for invoice #4567, amounting to $2750, by the end of this month.",
    "Invoice #8907 for the purchase of office supplies, total cost: $320.",
    "Please find attached invoice #1023 for $450, due on 10-12-2023.",
    "Invoice issued on 20-09-2023 for the service charge of $200.",
    "Invoice #3421 – Total: $1,200, Payment due by 05-11-2023.",
    "Payment for invoice #9876 received on 29-09-2023 for $3,750.",
    "Final notice for invoice #1452, total due: $875, please pay immediately.",
    "Invoice #5678 from Pizza Hut for catering services, amount: $900.",

    # Contract examples
    "The service contract begins on 01-10-2023 and runs for 12 months.",
    "Contract #2345 signed on 15-09-2023, total value: $20,000.",
    "This agreement is for a 6-month term, starting from 01-11-2023.",
    "Please review the attached contract for the supply of goods.",
    "This contract is non-cancellable after the first 3 months.",
    "The contractor will be paid $50 per hour as per the agreement.",
    "This contract requires delivery of services by 01-12-2023.",
    "The buyer agrees to purchase services as detailed in Schedule 1 of this contract.",
    "Contract #6789 – Signed on 10-10-2023, total cost: $5000.",
    "The contract is subject to force majeure and other conditions as listed."
]

# Preprocess function
def preprocess_text(text):
    review = re.sub('[^a-zA-Z\']', ' ', text)  # Keep apostrophes
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    return review

# Predict for each text
for text in test_texts:
    processed_text = preprocess_text(text)
    X_custom = vectorizer.transform([processed_text])

    # Check if classifier is fitted, if not, fit it
    if not hasattr(classifier, 'estimators_'):
        classifier.fit(X_train, y_train)

    # Predict class label using the trained classifier
    predicted_class = classifier.predict(X_custom)[0]

    # Print the predicted class
    print(f"Text: '{text}' -> Predicted class: {predicted_class}")


Text: 'Invoice #12345 issued on 15-09-2023 for $5000.' -> Predicted class: Invoice
Text: 'Payment due for invoice #789 on 01-11-2023. Total amount: $1500.' -> Predicted class: Invoice
Text: 'Kindly remit payment for invoice #4567, amounting to $2750, by the end of this month.' -> Predicted class: Invoice
Text: 'Invoice #8907 for the purchase of office supplies, total cost: $320.' -> Predicted class: Invoice
Text: 'Please find attached invoice #1023 for $450, due on 10-12-2023.' -> Predicted class: Invoice
Text: 'Invoice issued on 20-09-2023 for the service charge of $200.' -> Predicted class: Invoice
Text: 'Invoice #3421 – Total: $1,200, Payment due by 05-11-2023.' -> Predicted class: Invoice
Text: 'Payment for invoice #9876 received on 29-09-2023 for $3,750.' -> Predicted class: Invoice
Text: 'Final notice for invoice #1452, total due: $875, please pay immediately.' -> Predicted class: Invoice
Text: 'Invoice #5678 from Pizza Hut for catering services, amount: $900.' -> Predicted class

# Extracting image(invoice or contract) text


In [ ]:
import re
import pytesseract
import cv2
from PIL import Image

# Load the image using OpenCV
img_path = '/content/Contract.png'  # Update with your image path
img = cv2.imread(img_path)

# Preprocess the image (convert to grayscale and apply thresholding)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Extract text using Tesseract
extracted_text = pytesseract.image_to_string(thresh)

# print(extracted_text)

In [ ]:
processed_text = preprocess_text(extracted_text)
X_custom = vectorizer.transform([processed_text])

# Check if classifier is fitted, if not, fit it
if not hasattr(classifier, 'estimators_'):
    classifier.fit(X_train, y_train)

# Predict class label using the trained classifier
predicted_class = classifier.predict(X_custom)[0]

# Print the predicted class
print(f"Text: '{text}' -> Predicted class: {predicted_class}")

Text: 'The contract is subject to force majeure and other conditions as listed.' -> Predicted class: Contract


In [13]:
import joblib
joblib.dump(classifier, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']